In [1]:
import sys
import importlib
import datetime

import numpy as np
import pandas as pd
import xarray as xr
import pvlib as pv
import matplotlib.pyplot as plt
import scipy.ndimage.filters as filters

In [2]:
%matplotlib inline

In [3]:
sys.path.append('/home/travis/python_code/letkf_forecasting/')
import letkf_forecasting as lf
import prepare_sat_data as prep

In [5]:
importlib.reload(prep)

<module 'prepare_sat_data' from '/home2/travis/python_code/letkf_forecasting/prepare_sat_data.py'>

In [6]:
sat_data = xr.open_dataset('/a2/uaren/goes_images/crop/sat_images.nc')

In [7]:
elevation_data = xr.open_dataset(
    '/home/travis/python_code/letkf_forecasting/elevation_data.nc')

In [8]:
tus = pv.location.Location(32.2, -111, 'US/Arizona', 700, 'Tucson')

In [9]:
midday_sat = prep.midday(sat_data, tus, 60)

In [10]:
midday_sat.to_netcdf('midday_interp_sat.nc')

In [11]:
midday_sat = xr.open_dataset('./midday_interp_sat.nc')

In [12]:
this = midday_sat.data.values
shape = this.shape
these_times = pd.to_datetime(
    midday_sat.time).tz_localize('UTC').tz_convert('MST')
midday_sat_dataframe = pd.DataFrame(this.reshape(shape[0], shape[1]*shape[2]), index=these_times)

In [13]:
midday_sat_dataframe.to_hdf('./data/pixel.h5', 'pixel')

In [14]:
domain_shape = shape[1:]
np.save('./data/domain_shape', domain_shape)

In [15]:
elevation = elevation_data.elevation.values.ravel()
lats = midday_sat.lat.values.ravel()
lons = midday_sat.lon.values.ravel()
x = midday_sat.x.values.ravel()
y = midday_sat.y.values.ravel()
np.save('./data/elevation', elevation)
np.save('./data/lats', lats)
np.save('./data/lons', lons)
np.save('./data/x', x)
np.save('./data/y', y)

In [16]:
pixel = pd.read_hdf('./data/pixel.h5')
pixel.index = pixel.index.tz_convert('MST')
domain_shape = np.load('./data/domain_shape.npy')
elevation = np.load('./data/elevation.npy')
x = np.load('./data/x.npy')
y = np.load('./data/y.npy')
lats = np.load('./data/lats.npy')
lons = np.load('./data/lons.npy')

In [18]:
importlib.reload(prep)
low, up, cloudiness_index = prep.get_cloudiness_index(pixel, lats, lons, elevation)

In [21]:
np.save('./data/low', low)
np.save('./data/up', up)
cloudiness_index.to_hdf(
    './data/cloudiness_index.h5', 'cloudiness_index')

In [ ]:
cloudiness_index2 = pd.read_hdf('./data/cloudiness_index.h5')

In [ ]:
cloudiness_index

In [ ]:
cloudiness_index.index

In [ ]:
cloudiness_index.index[0]

In [ ]:
cloudiness_index.index = cloudiness_index.index.tz_convert('MST')

In [ ]:
cloudiness_index.index[0]

In [ ]:
cloudiness_index.shape

In [ ]:
cloudiness_index.to_hdf('./data/cloudiness_index.h5', 'cloudiness_index')

In [ ]:
low = np.load('./data/low.npy')
up = np.load('./data/up.npy')
cloudiness_index = pd.read_hdf('./data/cloudiness_index.h5')

In [ ]:
importlib.reload(prep)
clearsky = prep.get_clearsky(pixel.index, elevation, lats, lons)

In [ ]:
clearsky.to_hdf('./data/clearsky.h5', 'clearsky')

In [ ]:
clearsky.shape

In [ ]:
this_time = pd.Timestamp('2014-04-15 12:45:00').tz_localize('MST')
CI = cloudiness_index.ix[this_time].reshape(domain_shape).copy()
ktm = 2.36*CI**5 - 6.2*CI**4 + 6.22*CI**3 - 2.63*CI**2 - 0.58*CI + 1
Ghcnew = clearsky.ix[this_time].reshape(domain_shape).copy()
GHI = ktm*Ghcnew*(0.0001*ktm*Ghcnew + 0.9)

In [ ]:
this_time = pd.Timestamp('2014-04-14 12:45:00').tz_localize('MST')
CI_c = cloudiness_index.ix[this_time].reshape(domain_shape).copy()
ktm_c = 2.36*CI_c**5 - 6.2*CI_c**4 + 6.22*CI_c**3 - 2.63*CI_c**2 - 0.58*CI_c + 1
Ghcnew_c = clearsky.ix[this_time].reshape(domain_shape).copy()
GHI_c = ktm_c*Ghcnew_c*(0.0001*ktm_c*Ghcnew_c + 0.9)

In [ ]:
plt.figure()
this_image = GHI/GHI_c
print(this_image.max() - this_image.min())
im = plt.pcolormesh(
    this_image,
    cmap='Blues',
    vmin=0, vmax=1)
plt.colorbar(im)

plt.figure()
this_image = GHI/Ghcnew
print(this_image.max() - this_image.min())
im = plt.pcolormesh(
    this_image,
    cmap='Blues',
    vmin=0, vmax=1)
plt.colorbar(im)